In [1]:
sys.executable

'/home/goonieb/anaconda2/bin/python'

In [2]:
import os
import json
import pymongo
import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser

In [3]:
# setup mongo
client = MongoClient()
db = client["bandstour"]
bandsintown =  db["bandsintown"]


#setup mbrainz client
musicbrainzngs.set_useragent("Example music app", "0.1", "http://example.com/music")

# source data
DATA_FOLDER = "data/labels"
countMBID = 0
countNAME = 0
count = 0
# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'
# init label_db_retrieval
labels_aliases ={}
#retrieve label for each artist before a tour starts
#i.e what is the album previously to the first year
for artist in db.minedArtists.find() :
                    print artist["name"]
    
                    #print db.minedArtists[artist]
                    #print artist
                    labels_dict= defaultdict(list)
                    labels_dict["id"]=[]

                    if  artist["mbid"] is ( None ) :
                        print ("NOMBID for",artist["name"])

                    else:
                     if  bool(artist["mbid"]) is ( False ) :
                      print ("NOMBID2 for",artist["name"])

                     else:
                      print "aartist_mbid", artist["mbid"]
                      try:
                        artist["releases"]=(musicbrainzngs.browse_releases( artist["mbid"], release_type=["album"], includes=["labels"] ))
                            
                                  
                        #print "...."
                        #print artist["release_groups"]
                        key= 0
                        for release in  artist["releases"]["release-list"]:
                            #print release
                            if  release.has_key("release-event-list"):
                                #print "artist_name:",artist["name"]
                                #print "albumtitle:",release["title"]
                                #print "albumid:",release["id"]
                                #print "label-list",release["label-info-list"]
                                #print "releaseLBL",release["label-info-list"]
                                if not release["label-info-list"]:
                                    print "pppppppppppppp"
                                
                                else:
                                 if release["label-info-list"][0].has_key('label'):
                                  for label in release["label-info-list"]:
                                    #WE NEED TO TEST HERE
                                    #print "label:",label["label"]["sort-name"]
                                    #print "labelid:",label
                                    
#WE CREATE A DICTIONNARY WITH OUR FIELDS, WITH FORMAT
                                    if label.has_key('label'):
                                     if label["label"]["id"] not in labels_dict["labids"]:
                                        labels_dict["labids"].append(label["label"]["id"])
                                        labels_dict[label["label"]["id"]]={}

                                        labels_dict[label["label"]["id"]]["label_name"]=label["label"]["sort-name"]
                                        labels_dict[label["label"]["id"]]["label_id" ]=label["label"]["id"]
#WE Build the label aliases tables here so that all labels have their relationships##
                                        if label["label"]["id"] not in labels_aliases:
                                            labels_aliases[label["label"]["id"]] = musicbrainzngs.get_label_by_id(label["label"]["id"], includes=["aliases", "annotation", "area-rels", "label-rels", "place-rels"])
                                        #print "labels_aliases",labels_aliases
                                     labels_dict[label["label"]["id"]][str(key)]={}
                                    #print key
                                     labels_dict[label["label"]["id"]][str(key)]["album_id"]=release["id"]
                                     labels_dict[label["label"]["id"]][str(key)]["album_title"]=release["title"]
                                     releDate=release["release-event-list"][0]
                                #print "releDate", release["release-event-list"][0]
                                     if  releDate.has_key("date"):
                                        dateRel=arrow.get(releDate["date"])
                                        #print "date:", dateRel.strftime('%Y-%m-%d')
                                        labels_dict[label["label"]["id"]][str(key)]["album_date"]=dateRel.strftime('%Y-%m-%d')
                                     key = key +1
                                 else:
                                    print "NORELEASEDATE2"
                                    #key = key +1
                            #else:
                                #key = key +1
                                #print "NORELEASEDATE"
#NOW we make a nice lightweight object to push to mongo
                        labels_dict["name"]=artist["name"]
                        print labels_dict["name"],'\r\n'
                        db.labels_dict_per_artist.update({'myid': artist["name"] } ,labels_dict, upsert=True, multi=False)
                      except  musicbrainzngs.ResponseError:
                        print "oups",artist["name"]

                #with open(os.path.join(DATA_FOLDER,artist["name"],".json"), 'w') as outfile:
                             #   json.dump(labels_dict, outfile)
                            #db.minedArtists.artist.insert_one(labels_dict)
#with open(os.path.join(DATA_FOLDER,"label_equiv_table.json"), 'w') as outfile:
 #                               json.dump(labels_aliases, outfile)
#print labels_aliases,'\r\n\r\n\r\n'
import pickle



with open('labels_aliases', 'wb') as handle:
    pickle.dump(labels_aliases, handle, protocol=pickle.HIGHEST_PROTOCOL)



조수미
aartist_mbid dd21a142-cf44-4e22-add6-a50eea696854
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
조수미 

Лили Иванова
aartist_mbid ab634a19-20e3-4801-975e-a230f899af8a


/home/goonieb/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:94: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


Лили Иванова 

ᵀᴴᴱ ᴼᴿᴵᴳᴵᴻᴬᴸ Funky Monkey
('NOMBID for', u'\u1d40\u1d34\u1d31 \u1d3c\u1d3f\u1d35\u1d33\u1d35\u1d3b\u1d2c\u1d38 Funky Monkey')
Калевала
('NOMBID for', u'\u041a\u0430\u043b\u0435\u0432\u0430\u043b\u0430')
Кристина Орбакайте
aartist_mbid e71738bd-39d4-4699-b6c2-688f95b31732
pppppppppppppp
pppppppppppppp
Кристина Орбакайте 

Дмитрий Дмитриевич Шостакович
aartist_mbid 519dd32e-8f30-4380-8826-7aa99169e1bb
Дмитрий Дмитриевич Шостакович 

Валерия
aartist_mbid b4313680-2f57-4ec6-87a7-24873133ba4c
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
Валерия 

Евгений Игоревич Кисин
aartist_mbid 5761282a-0965-4ff6-9796-7b9ba0ed70e2


KeyboardInterrupt: 